# **La bagarre**

---



## Preamble

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

### Global variables

In [1]:
batch_size = 512
data_path = './data'
input_shape = (224,224,3)
num_classes = 6

### Imports

In [2]:
import os
import numpy as np
#import cv2
import PIL

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping 
from tensorflow.keras.applications import NASNetMobile
from sklearn.metrics import confusion_matrix, classification_report

import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm

In [3]:
tf.__version__
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Functions

In [ ]:
def random_images(n):
    images = []
    i = 1
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            images.append(os.path.join(dirname, filename))
    samples = np.random.randint(len(filenames), size=n)
    print(f'Displaying {n} images from a total of {len(images)} found.')
    f = plt.figure(figsize=(2*n,15))
    for sample in samples:
        sample_image = cv2.imread(images[sample])
        f.add_subplot(1, n, i)
        plt.imshow(sample_image)
        i = i + 1

## Data

In [ ]:
random_images(8)

In [ ]:
#!pip install split-folders
import splitfolders as sf

sf.ratio(f'{data_path}/raw',
  output=f'{data_path}/processed',
  seed=42,
  ratio=(.8, .1, .1),
  group_prefix=None,
  move=False)

In [4]:
idg_aug = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

idg_raw = ImageDataGenerator(rescale=1./255)


train = idg_aug.flow_from_directory(f'{data_path}/processed/train',
                                subset='training',
                                target_size=(331, 331),
                                class_mode="categorical",
                                batch_size=batch_size,
                                shuffle=False,
                                seed=42)

val = idg_raw.flow_from_directory(f'{data_path}/processed/val',
                                subset='training',
                                target_size=(331, 331),
                                class_mode="categorical",
                                shuffle=False,
                                seed=42)

test = idg_raw.flow_from_directory(f'{data_path}/processed/test',
                                subset='training',
                                target_size=(331, 331),
                                class_mode="categorical",
                                shuffle=False,
                                seed=42)

Found 37734 images belonging to 6 classes.
Found 4716 images belonging to 6 classes.
Found 4719 images belonging to 6 classes.


## Train Model

In [5]:
# Reduce LR
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.00001)

# Checkpoints
checkpoint_filepath = "./model/checkpoint"
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,
                             monitor='val_loss', verbose=1,
                             save_weights_only=True,
                             save_best_only=True, mode='min')

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=5,
                           restore_best_weights=True,
                           mode='min')

callbacks = [checkpoint, early_stop, reduce_lr]

In [6]:
pre_trained_model = NASNetMobile(include_top=False, pooling='avg', input_shape=input_shape)
pre_trained_model.trainable = False


for layer in pre_trained_model.layers[:-3]:
    layer.trainable = True

Metal device set to: Apple M1


2022-03-29 15:12:04.569734: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-29 15:12:04.570197: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:
print(pre_trained_model.summary())

Model: "NASNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 111, 111, 32  128         ['stem_conv1[0][0]']             
                                )                                                            

                                                                                                  
 separable_conv_1_pad_reduction  (None, 115, 115, 32  0          ['activation_267[0][0]']         
 _right3_stem_1 (ZeroPadding2D)  )                                                                
                                                                                                  
 activation_269 (Activation)    (None, 56, 56, 11)   0           ['reduction_add_1_stem_1[0][0]'] 
                                                                                                  
 separable_conv_1_bn_reduction_  (None, 56, 56, 11)  44          ['separable_conv_1_reduction_righ
 right2_stem_1 (BatchNormalizat                                  t2_stem_1[0][0]']                
 ion)                                                                                             
                                                                                                  
 separable

 add_4 (Add)                    (None, 56, 56, 11)   0           ['reduction_add_2_stem_1[0][0]', 
                                                                  'reduction_left4_stem_1[0][0]'] 
                                                                                                  
 reduction_add4_stem_1 (Add)    (None, 56, 56, 11)   0           ['separable_conv_2_bn_reduction_l
                                                                 eft4_stem_1[0][0]',              
                                                                  'reduction_right5_stem_1[0][0]']
                                                                                                  
 cropping2d_4 (Cropping2D)      (None, 111, 111, 32  0           ['zero_padding2d_4[0][0]']       
                                )                                                                 
                                                                                                  
 reduction

 separable_conv_2_bn_reduction_  (None, 28, 28, 22)  88          ['separable_conv_2_reduction_righ
 right1_stem_2 (BatchNormalizat                                  t1_stem_2[0][0]']                
 ion)                                                                                             
                                                                                                  
 separable_conv_1_pad_reduction  (None, 61, 61, 22)  0           ['activation_276[0][0]']         
 _right2_stem_2 (ZeroPadding2D)                                                                   
                                                                                                  
 activation_278 (Activation)    (None, 56, 56, 22)   0           ['adjust_bn_stem_2[0][0]']       
                                                                                                  
 reduction_add_1_stem_2 (Add)   (None, 28, 28, 22)   0           ['separable_conv_2_bn_reduction_l
          

                                                                                                  
 separable_conv_2_bn_reduction_  (None, 28, 28, 22)  88          ['separable_conv_2_reduction_left
 left4_stem_2 (BatchNormalizati                                  4_stem_2[0][0]']                 
 on)                                                                                              
                                                                                                  
 reduction_right5_stem_2 (MaxPo  (None, 28, 28, 22)  0           ['reduction_pad_1_stem_2[0][0]'] 
 oling2D)                                                                                         
                                                                                                  
 zero_padding2d_5 (ZeroPadding2  (None, 57, 57, 44)  0           ['adjust_relu_1_0[0][0]']        
 D)                                                                                               
          

...

 separable_conv_1_normal_left1_  (None, 7, 7, 176)   35376       ['activation_413[0][0]']         
 10 (SeparableConv2D)                                                                             
                                                                                                  
 separable_conv_1_normal_right1  (None, 7, 7, 176)   32560       ['activation_415[0][0]']         
 _10 (SeparableConv2D)                                                                            
                                                                                                  
 separable_conv_1_normal_left2_  (None, 7, 7, 176)   35376       ['activation_417[0][0]']         
 10 (SeparableConv2D)                                                                             
                                                                                                  
 separable_conv_1_normal_right2  (None, 7, 7, 176)   32560       ['activation_419[0][0]']         
 _10 (Sepa

                                                                                                  
 normal_add_1_10 (Add)          (None, 7, 7, 176)    0           ['separable_conv_2_bn_normal_left
                                                                 1_10[0][0]',                     
                                                                  'separable_conv_2_bn_normal_righ
                                                                 t1_10[0][0]']                    
                                                                                                  
 normal_add_2_10 (Add)          (None, 7, 7, 176)    0           ['separable_conv_2_bn_normal_left
                                                                 2_10[0][0]',                     
                                                                  'separable_conv_2_bn_normal_righ
                                                                 t2_10[0][0]']                    
          

 activation_426 (Activation)    (None, 7, 7, 176)    0           ['separable_conv_1_bn_normal_left
                                                                 1_11[0][0]']                     
                                                                                                  
 activation_428 (Activation)    (None, 7, 7, 176)    0           ['separable_conv_1_bn_normal_righ
                                                                 t1_11[0][0]']                    
                                                                                                  
 activation_430 (Activation)    (None, 7, 7, 176)    0           ['separable_conv_1_bn_normal_left
                                                                 2_11[0][0]']                     
                                                                                                  
 activation_432 (Activation)    (None, 7, 7, 176)    0           ['separable_conv_1_bn_normal_righ
          

 activation_436 (Activation)    (None, 7, 7, 1056)   0           ['normal_concat_11[0][0]']       
                                                                                                  
 adjust_conv_projection_12 (Con  (None, 7, 7, 176)   185856      ['activation_435[0][0]']         
 v2D)                                                                                             
                                                                                                  
 normal_conv_1_12 (Conv2D)      (None, 7, 7, 176)    185856      ['activation_436[0][0]']         
                                                                                                  
 adjust_bn_12 (BatchNormalizati  (None, 7, 7, 176)   704         ['adjust_conv_projection_12[0][0]
 on)                                                             ']                               
                                                                                                  
 normal_bn

 separable_conv_2_bn_normal_lef  (None, 7, 7, 176)   704         ['separable_conv_2_normal_left1_1
 t1_12 (BatchNormalization)                                      2[0][0]']                        
                                                                                                  
 separable_conv_2_bn_normal_rig  (None, 7, 7, 176)   704         ['separable_conv_2_normal_right1_
 ht1_12 (BatchNormalization)                                     12[0][0]']                       
                                                                                                  
 separable_conv_2_bn_normal_lef  (None, 7, 7, 176)   704         ['separable_conv_2_normal_left2_1
 t2_12 (BatchNormalization)                                      2[0][0]']                        
                                                                                                  
 separable_conv_2_bn_normal_rig  (None, 7, 7, 176)   704         ['separable_conv_2_normal_right2_
 ht2_12 (B

In [7]:
clf_model = tf.keras.models.Sequential()
clf_model.add(pre_trained_model)
clf_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
clf_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
#clf_model.load_weights(checkpoint_filepath)
history = clf_model.fit(train, callbacks=callbacks, epochs=20, validation_data=val)

Epoch 1/20


2022-03-29 15:12:37.584777: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-29 15:12:42.811453: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/NASNet/normal_add_3_0/add' defined at (most recent call last):
    File "/Users/lss/.pyenv/versions/miniforge3-4.10/envs/tf-metal/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/lss/.pyenv/versions/miniforge3-4.10/envs/tf-metal/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/lss/.pyenv/versions/miniforge3-4.10/envs/tf-metal/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/Users/lss/.pyenv/versions/miniforge3-4.10/envs/tf-metal/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/lss/.pyenv/versions/miniforge3-4.10/envs/tf-metal/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/Users/lss/.pyenv/ve

In [ ]:
clf_model.save('./model/bagarre-2')

## Evaluate Model

In [9]:
model = tf.keras.models.load_model('./model/NASNetMobile-mednist')
y_pred = model.predict(test)
y_pred = np.argmax(y_pred, axis=1)

2022-03-29 15:15:13.619600: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_9/NASNet/normal_add_3_0/add' defined at (most recent call last):
    File "/Users/lss/.pyenv/versions/miniforge3-4.10/envs/tf-metal/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/lss/.pyenv/versions/miniforge3-4.10/envs/tf-metal/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/lss/.pyenv/versions/miniforge3-4.10/envs/tf-metal/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/Users/lss/.pyenv/versions/miniforge3-4.10/envs/tf-metal/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/lss/.pyenv/versions/miniforge3-4.10/envs/tf-metal/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/Users/lss/.pyenv/

In [21]:
print('Confusion Matrix')
print(confusion_matrix(test.classes, y_pred))

Confusion Matrix
[[800   0   0   0   0   0]
 [  0 717   0   0   0   0]
 [  0   0 800   0   0   0]
 [  0   0   0 800   0   0]
 [  0   0   0   0 800   0]
 [  0   0   0   0   0 802]]


In [22]:
print('Classification Report')
print(classification_report(test.classes, y_pred))

Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       800
           1       1.00      1.00      1.00       717
           2       1.00      1.00      1.00       800
           3       1.00      1.00      1.00       800
           4       1.00      1.00      1.00       800
           5       1.00      1.00      1.00       802

    accuracy                           1.00      4719
   macro avg       1.00      1.00      1.00      4719
weighted avg       1.00      1.00      1.00      4719

